# Objective

This is to show and compare local runs for the `Ai-for-Astronmy` lab, concerned with
* different batch sizes
* different data sizes
* any starting graphs for a baseline

There are different machines on our team, try to show different machine info

## This to try

###  batch sizes
* batch sizes to try: small 8, 16, 32, 64
* batch sizes to try: medium 128, 256
* batch sizes to try: small


In [1]:
# !pip install tensorflow
# not strictly necessary consider only for notebook or adding to requirements.txt[dev]

# make sure astronomy for ai is updated
# !cd ..// && make update_Astronomy_AI

In [2]:
import sys

import boto3
import platform
import psutil
import tensorflow as tf
import yaml

with open("../keys/aws_credentials.yaml", "r") as f:
    credentials = yaml.safe_load(f)


print(sys.version)
print(tf.__version__)


3.11.8 (v3.11.8:db85d51d3e, Feb  6 2024, 18:02:37) [Clang 13.0.0 (clang-1300.0.29.30)]
2.18.0


In [3]:
print(platform.system()) # mac - Darwin
print(platform.processor()) # cpu

# Get system memory info
memory = psutil.virtual_memory()
print(f"Total Memory: {memory.total / (1024 ** 3):.2f} GB")
print(f"Available Memory: {memory.available / (1024 ** 3):.2f} GB")
print(f"Used Memory: {memory.used / (1024 ** 3):.2f} GB")
print(f"Free Memory: {memory.free / (1024 ** 3):.2f} GB")

devices = tf.config.list_physical_devices()
print(devices) # only cpu on local


Darwin
arm
Total Memory: 48.00 GB
Available Memory: 21.48 GB
Used Memory: 15.89 GB
Free Memory: 6.35 GB
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


#

In [4]:
# Initialize a session using aws cred
session = boto3.Session(aws_access_key_id=credentials["aws_access_key_id"],
                        aws_secret_access_key=credentials["aws_secret_access_key"],
                        region_name=credentials["region"])

s3 = session.client('s3')

d = s3.list_buckets()

# show current buckets
b = [n["Name"] for n in d["Buckets"]]

b

['aws-athena-query-results-211125778552-us-east-1',
 'cosmicai-data',
 'cosmicai2',
 'fmi-lambda-demo',
 'group2-s3-bucket',
 'group4-s3-bucket',
 'sagemaker-studio-211125778552-3zpozdpwzcx',
 'sagemaker-studio-211125778552-rrp76qgcj1n',
 'sagemaker-us-east-1-211125778552',
 'team-one-cosmic-data',
 'team-one-s3-cosmic',
 'team2cosmicai',
 'team3cosmicai',
 'team4-cosmicai']

In [5]:
'cosmicai-data'

'cosmicai-data'

In [6]:
from cosmicai.inference import CosmicAI
import warnings

warnings.filterwarnings("ignore")


cosmic = CosmicAI()
cosmic.load_data()
cosmic.load_model()

loading data from - ../data/raw/resized_inference.pt
loading the model from  ../data/models/Mixed_Inception_z_VITAE_Base_Img_Full_New_Full.pt


In [7]:
# !pip install torch-summary not required just visualizing
# this is a keras style visualizer

from torchsummary import summary

summary(cosmic.model)

Layer (type:depth-idx)                   Param #
├─LayerNorm: 1-1                         384
├─Linear: 1-2                            37,056
├─Input_Mix_Model: 1-3                   --
|    └─Conv2d: 2-1                       192
|    └─AvgPool2d: 2-2                    --
|    └─AvgPool2d: 2-3                    --
|    └─Sequential: 2-4                   --
|    |    └─Linear: 3-1                  6,144
|    |    └─ReLU: 3-2                    --
|    |    └─Linear: 3-3                  1,049,600
|    |    └─ReLU: 3-4                    --
|    |    └─Linear: 3-5                  1,049,600
|    |    └─ReLU: 3-6                    --
|    |    └─Linear: 3-7                  1,049,600
|    |    └─ReLU: 3-8                    --
|    |    └─Linear: 3-9                  1,049,600
|    └─Sequential: 2-5                   --
|    |    └─Conv2d: 3-10                 (recursive)
|    |    └─AvgPool2d: 3-11              --
|    |    └─InceptionBlock: 3-12         11,040
|    |    └─Inception

Layer (type:depth-idx)                   Param #
├─LayerNorm: 1-1                         384
├─Linear: 1-2                            37,056
├─Input_Mix_Model: 1-3                   --
|    └─Conv2d: 2-1                       192
|    └─AvgPool2d: 2-2                    --
|    └─AvgPool2d: 2-3                    --
|    └─Sequential: 2-4                   --
|    |    └─Linear: 3-1                  6,144
|    |    └─ReLU: 3-2                    --
|    |    └─Linear: 3-3                  1,049,600
|    |    └─ReLU: 3-4                    --
|    |    └─Linear: 3-5                  1,049,600
|    |    └─ReLU: 3-6                    --
|    |    └─Linear: 3-7                  1,049,600
|    |    └─ReLU: 3-8                    --
|    |    └─Linear: 3-9                  1,049,600
|    └─Sequential: 2-5                   --
|    |    └─Conv2d: 3-10                 (recursive)
|    |    └─AvgPool2d: 3-11              --
|    |    └─InceptionBlock: 3-12         11,040
|    |    └─Inception

In [8]:
cosmic.run_inference()

{'total_cpu_time': 14.384935184999893,
 'total_cpu_memory': 29616.327912,
 'execution_time_per_batch': 4.794978394999965,
 'num_batches': 3,
 'batch_size': 512,
 'device': 'cpu',
 'throughput_bps': 14299159.318735681}

In [10]:
# 1) download data 
    # data should be stored in raw and can't be uploaded

# 2) run the model on the data

# 3) set parameters for the run and collect them